In [1]:
import openai
import json
from tqdm import tqdm
import os
import dotenv
import time
dotenv.load_dotenv()

True

In [2]:
openai.api_key = os.environ.get('OPENAI_KEY')

In [3]:
with open('../generations/btlm_beta0.3_shp-temp1.json', 'r') as f:
    generations = json.load(f)

In [4]:
with open('../generations/prompt.txt', 'r') as f:
    prompt = f.read()

In [5]:
gpt_responses = []

In [36]:
for i, generation in tqdm(enumerate(generations), total=len(generations)):
    policy_first = True
    responseA = generation['policy_response']
    responseB = generation['chosen_response']
    # change the order every time for more reliable result
    if i%2 == 0:
        responseA, responseB = responseB, responseA
        policy_first = False

    content = prompt.format(user_query=generation['prompt'], responseA=responseA, responseB=responseB)
    chatgpt_eval = openai.ChatCompletion.create(model="gpt-4", messages=[
            {"role": "user", "content": content},
        ])
    chat_eval = chatgpt_eval.choices[0].message.content

    gpt_responses.append({
        "prompt": content,
        "user_query": generation['prompt'],
        "policy_first": policy_first,
        "policy_response": generation['policy_response'],
        "chosen_response": generation['chosen_response'],
        "chatgpt_eval": chat_eval,
        "only_answer": chat_eval[chat_eval.rfind('More helpful'):]
    })
    time.sleep(0.1)

100%|██████████| 256/256 [32:30<00:00,  7.62s/it]


In [8]:
gpt_responses = []
directory_path = 'btlm_beta0.3_shp'

for name in os.listdir(directory_path):
    with open(os.path.join(directory_path, name), 'r') as f:
        gpt_response = json.loads(f.read())
        gpt_responses.append(gpt_response)

In [9]:
len(gpt_responses)

256

In [38]:
with open('../generations/gpt_eval-btlm_beta0.1_shp-temp1.json', 'w') as json_file:
    json.dump(gpt_responses, json_file)

In [17]:
with open('../generations/gpt_eval-temp1.json', 'r') as json_file:
    gpt_responses = json.load(json_file)

In [11]:
count = len(gpt_responses)
hit = 0

for resp in gpt_responses:
    hit += (resp['policy_first'] and resp['only_answer'][-1] == 'A') or (not resp['policy_first'] and resp['only_answer'][-1] == 'B')
print(f"Win rate: {hit/count}")

Win rate: 0.3984375


In [76]:
# generations_ = []

In [77]:
# for gen in generations:
#     assistant_word = '\n\nAssistant:'
#     resp_indx = gen['prompt'][0].rfind(assistant_word)
#     prompt = gen['prompt'][0][:resp_indx]
#     chosen_response = gen['chosen_response'][0][resp_indx+len(assistant_word):].strip()
#     policy_response = gen['policy_response'][0][resp_indx+len(assistant_word):].strip()
#     reference_response = gen['reference_response'][0][resp_indx+len(assistant_word):].strip()
#     generations_.append({'prompt': prompt, 'chosen_response': chosen_response, 'policy_response': policy_response, 'reference_response': reference_response})

In [79]:
# generations = generations_

In [ ]:
gpt_responses

In [ ]:
# Pythia without LoRa float32: 0.46 (Probably something wrong)
# Pythia without LoRa bfloat16: 0.73
# Original authors model: 0.707
# BTLM beta 0.1 LoRa HH: 0.74  rewards/accuracies: 0.6296
# BTLM beta 0.3 LoRa HH: 0.59  rewards/accuracies: 0.6166
# BTLM beta 0.5 LoRa HH: 0.52  rewards/accuracies: 0.6092
# BTLM beta 0.1 LoRa SHP: 0.36
# BTLM beta 0.3 LoRa SHP: 0.40
# BTLM beta 0.5 LoRa SHP: 
# SFT BTLM: 0.43
# with LoRa and bfloat16: 

In [4]:
from glob import glob
import os

In [6]:
directory_path = 'btlm_beta0.1_shp'

for file in glob(f'{directory_path}/*'):
    try:
        os.remove(file)
    except OSError as e:
        print(f'Error: {file}: {e.strerror}')

In [7]:
import shutil

shutil.rmtree('btlm_beta0.1_shp')